kmp 2022

Based on **Lauwens & Downey "Think Julia: How to Think Like a Computer Scientist" 
https://benlauwens.github.io/ThinkJulia.jl/latest/book.html**

Resources:

Julia webpage https://julialang.org/ 

Julia documentation https://docs.julialang.org/en/v1/


## Chapter 14 -- Files

https://benlauwens.github.io/ThinkJulia.jl/latest/book.html#chap14

This chapter introduces the idea of persistent programs that keep data in permanent storage, and shows how to use different kinds of permanent storage, like files and databases.

https://docs.julialang.org/en/v1/base/file/

### Persistence

Most of the programs we have seen so far are transient in the sense that they run for a short time and produce some output, but when they end, their data disappears. If you run the program again, it starts with a clean slate.

Other programs are persistent: they run for a long time (or all the time); they keep at least some of their data in permanent storage (on a hard drive, for example); and if they shut down and restart, they pick up where they left off.

Examples of persistent programs are **operating systems**, which run pretty much whenever a computer is on, and **web servers**, which run all the time, waiting for requests to come in on the network.

One of the simplest ways for programs to maintain their data is by reading and writing **text files**. We have already seen programs that read text files; in this chapter we will see programs that write them. An alternative is to store the state of the program in a **database**. 

### Reading and Writing

A **text file is a sequence of characters stored on a permanent medium** like a hard drive. We saw how to open and read a file in Reading Word Lists. To write a file, you have to **`open`** it with **`mode "w"`** as a second parameter:

```Julia
	julia> fout = open("output.txt", "w")
		IOStream(<file output.txt>)
```

**Important to note: If the file already exists, opening it in write mode clears out the old data and starts fresh, so be careful!**

If the file does not exist, a new one is created. **`open`** returns a **file object** and the write function puts data into the file.

```Julia
	julia> line1 = "This here's the wattle,\n";

	julia> write(fout, line1)
		24
```

The return value is the number of characters that were written. The file object keeps track of where it is, so if you call write again, it adds the new data to the end of the file.

```Julia
	julia> line2 = "the emblem of our land.\n";

	julia> write(fout, line2)
		24
```

When you are done writing, you should **`close`** the file.

```Julia
	julia> close(fout)
```

**If you do not close the file, it gets closed for you when the program ends.**


### Formatting Text

The argument of write has to be a string, so if we want to put other values in a file, we have to convert them to strings. The easiest way to do that is with **string** or **string interpolation**:

```Julia
	julia> fout = open("output.txt", "a")
		IOStream(<file output.txt>)

	julia> write(fout, string(150))
		3
```

An alternative is to use the **print(ln)** family of functions.

```Julia
	julia> camels = 42
		42

	julia> println(fout, "I have spotted $camels camels.")
```

A more powerful alternative is the **`@printf` macro** that prints using a **C style format specification string**, which you can read about at https://docs.julialang.org/en/v1/stdlib/Printf/.

The **`Printf.@printf`** macro

Syntax: **`@printf([io::IO], "%Fmt", args...)`**

In [ ]:
using Printf

In [ ]:
@printf "Hello %s\n" "world!"

@printf "Scientific notation: %e\n" 1.234

@printf "Scientific notation three digits: %.3e\n" 1.23456

@printf "Decimal two digits: %.2f\n" 1.23456

@printf "Padded to length 5: %5i\n" 123

@printf "Padded with zeros to length: 6 %06i\n" 123

@printf "Use shorter of decimal or scientific: %g %g\n" 1.23 12300000.0

The **`Printf.@sprintf`** macro

Syntax: **`@sprintf("%Fmt", args...)`**

In [ ]:
@sprintf "this is a %s %15.1f" "test" 34.567

In [ ]:
str = @sprintf "this is a %s %15.1f" "test" 34.567;

In [ ]:
str

### Filenames and Paths

Files are organized into **directories** (or **folders**). Every running program has a **current directory**, which is the default directory for most operations. For example, when you open a file for reading, Julia looks for it in the current directory.

The function **`pwd()`** returns the name of the current directory:

```Julia
	julia> cwd = pwd()
		"/home/ben"
```

**`cwd`** stands for **current working directory**. The result in this example is _/home/ben_, which is the home directory of a user named ben.

In [ ]:
cd("E:\\aaa-Julia-course-2022\\lectures-1.8")
cwd = pwd()

In [ ]:
cwd

In [ ]:
cd()        # user or home directory
pwd()

In [ ]:
cd("..")    # step up once
pwd()

In [ ]:
cd(cwd)
pwd()

In [ ]:
cd("..\\..")    # step up twice
pwd()

In [ ]:
cd(cwd)
pwd()

In [ ]:
cd("\\")    # disk or partition root
pwd()

In [ ]:
cd(cwd)
pwd()

A string like "/home/ben" that identifies a file or directory is called a **path**. 

A simple filename, like `memo.txt` is also considered a path. It is a **relative path** because it relates to the current directory. If the current directory is /home/ben, then filename `memo.txt` refers to `/home/ben/memo.txt`, which is the **fullpath** or **absolute path** of `memo.txt`.

A path that begins with / does not depend on the current directory; it is called an **absolute path**. To find the absolute path to a file, you can use **abspath**:

In [ ]:
abspath("word.txt")

In [ ]:
ispath("word.txt")


Julia also provides other functions for working with filenames and paths. For example, **`ispath`** checks whether a file or directory exists:

```Julia
	julia> ispath("memo.txt")
		true
```

If it exists, **isdir** checks whether it is a directory:

```Julia
	julia> isdir("memo.txt")
		false

	julia> isdir("/home/ben")
		true
```

Similarly, **`isfile`** checks whether it is a file. **`readdir`** returns an array of the files (and other directories) in the given directory:

In [ ]:
isdir("memo.txt"), isdir(cwd)

In [ ]:
readdir()

To demonstrate these functions, the following example walks through a directory, prints the names of all the files, and calls itself recursively on all the directories.

In [ ]:
function walk(dirname)
    for name in readdir(dirname)
        path = joinpath(dirname, name)
        if isfile(path)
            println(path)
        else
            walk(path)
        end
    end
end

In [ ]:
walk(pwd())

**`joinpath`** takes a directory and a file name and joins them into a complete path.

Julia provides a function called **`walkdir`** that is similar to this one but more versatile. As an exercise, read the documentation and use it to print the names of the files in a given directory and its subdirectories.

https://docs.julialang.org/en/v1/base/file/#Base.Filesystem.walkdir

In [ ]:
path = "E:\\aaa-Julia-course-2022\\lectures-1.8\\Introduction-to-Julia\\notebooks"

for (root, dirs, files) in walkdir(path)

    println("Root directory: $root")

    if !isempty(dirs)
        println("Directories in $root")
        
        for dir in dirs
            println(joinpath(root, dir)) # path to directories
        end
    else 
        println("No directories in root directory.")
    end

    if !isempty(files)
        println("Files in root directory:")

        for file in files
            println(joinpath(root, file)) # path to files
        end
    end
end

### Catching Exceptions

A lot of things can go wrong when you try to read and write files. If you try to open a file that does not exist, you get a SystemError:

```Julia
	julia> fin = open("bad_file")
		ERROR: SystemError: opening file "bad_file": No such file or directory
```

If you do not have permission to access a file:

```Julia
	julia> fout = open("/etc/passwd", "w")
		ERROR: SystemError: opening file "/etc/passwd": Permission denied
```

To avoid these errors, you could use functions like **`ispath`** and **`isfile`**, but it would take a lot of time and code to check all the possibilities. It is easier to go ahead and try and deal with problems if they happen which is exactly what the **`try`** statement does. The syntax is similar to an if statement:


In [ ]:
fin = open("bad_file.txt")

In [ ]:
try
    fin = open("bad_file.txt")
catch exc
    println("Something went wrong: $exc")
end

Julia starts by executing the **`try clause`**. If all goes well, it skips the catch clause and proceeds. 

**If an exception occurs, it jumps out of the try clause and runs the catch clause**. 

Handling an exception with a try statement is called **`catching an exception`**. In this example, the except clause prints an error message that is not very helpful. In general, catching an exception gives you a chance to fix the problem, or try again, or at least end the program gracefully.

In code that performs state changes or uses resources like files, there is typically clean-up work (such as closing files) that needs to be done when the code is finished. Exceptions potentially complicate this task, since they can cause a block of code to exit before reaching its normal end. The **`finally`** keyword provides a way to run some code when a given block of code exits, regardless of how it exits:

```Julia
	f = open("output.txt")
	try
		line = readline(f)
		println(line)
	finally
		close(f)
	end
```
The function **`close`** will always be executed.

### Databases

A database is a file that is organized for storing data. Many databases are organized like a dictionary in the sense that they map from keys to values. The biggest difference between a database and a dictionary is that the database is on disk (or other **persistant storage**), so it persists after the program ends. **`ThinkJulia`** provides an interface to **`GDBM (GNU dbm)`** for creating and updating database files. As an example, we will create a database that contains captions for image files. Opening a database is similar to opening other files:

```Julia
	julia> using ThinkJulia

	julia> db = DBM("captions", "c")
		DBM(<captions>)
```

The **mode `"c"`** means that the database should be created if it does not already exist. The result is a **`database object`** that can be used (for most operations) like a dictionary. 

In [ ]:
using ThinkJulia
db = DBM("captions", "c")

When you create a new item, GDBM updates the database file:

```Julia
	julia> db["cleese.png"] = "Photo of John Cleese."
		"Photo of John Cleese."
```

In [ ]:
db["cleese.png"] = "Photo of John Cleese."

When you access one of the items, GDBM reads the file:

```Julia
	julia> db["cleese.png"]
		"Photo of John Cleese."
```

In [ ]:
db["cleese.png"]

If you make another assignment to an existing key, GDBM replaces the old value:

```Julia
	julia> db["cleese.png"] = "Photo of John Cleese doing a silly walk."
		"Photo of John Cleese doing a silly walk."

	julia> db["cleese.png"]
		"Photo of John Cleese doing a silly walk."
```

Some functions having a dictionary as argument, like keys and values, do not work with database objects. But iteration with a for loop works:

```Julia
	for (key, value) in db
		println(key, ": ", value)
	end
```


In [ ]:
db["cleese.png"] = "Photo of John Cleese doing a silly walk."

for (key, value) in db
    println(key, ": ", value)
end


As with other files, you should **close** the database when you are done:

In [ ]:
close(db)

### Serialization

A limitation of GDBM is that the keys and the values have to be **strings** or **byte arrays**. If you try to use any other type, you get an error. The functions **`serialize`** and **`deserialize`** can help. **They translate almost any type of object into a `byte array (an iobuffer)` suitable for storage in a database, and then translates byte arrays back into objects**:

In [ ]:
using Serialization

io = IOBuffer()

t = [1, 2, 3]
serialize(io, t)

print(take!(io))

The format is not obvious to human readers but is meant to be easy for Julia to interpret. 

**deserialize** reconstitutes the object:

In [ ]:
io = IOBuffer()
t1 = [1, 2, 3]
serialize(io, t1)

s = take!(io)
t2 = deserialize(IOBuffer(s))

`serialize` and `deserialize` write to and read from a **`iobuffer object`** which represents an **in-memory I/O stream**. The function **`take!` fetches** the contents of the iobuffer as a **`byte array`** and **resets the iobuffer to its initial state**. Although the new object has the same value as the old, it is not (in general) the same object:

In [ ]:
t1 == t2, t1 ≡ t2

**In other words, serialization and then deserialization has the same effect as copying the object.** You can use this to store non-strings in a database. In fact, the storage of non-string in a database is common and has been encapsulated in a package called **`JLD2`** (see https://github.com/JuliaIO/JLD2.jl).

### Modules and File Inclusion

Suppose you have a file named "wc.jl" with the following code:

```Julia
	function linecount(filename)
		count = 0
		for line in eachline(filename)
			count += 1
		end
		count
	end

	print(linecount("wc.jl"))
```

If you run this program, it reads itself and prints the number of lines in the file, which is 9. 

You can also **`include`** it in the REPL like this:

```Julia
	julia> include("wc.jl")
		9
```

Julia introduces **modules** to create **`separate variable workspaces`**, i.e., **`new global scopes`**. 

A module starts with the keyword **`module`** and ends with **`end`**. 

Naming conflicts are avoided between your own top-level definitions and those found in somebody else’s code. 

**`import`** allows to control which names from other modules are visible and **`export`** specifies which of your names are public, i.e., **can be used outside the module without being prefixed with the name of the module**.

```Julia
	module LineCount

		export linecount

		function linecount(filename)
		   count = 0
		   for line in eachline(filename)
				count += 1
		   end
		   return count
		end

	end
```

The module LineCount object provides linecount:

```Julia
	julia> using LineCount

	julia> linecount("wc.jl")
		11
```

### Exercise 14-1

Type this example into a file named wc.jl, include it into the REPL and enter using LineCount. If you import a module that has already been imported, Julia does nothing. **It does not re-read the file, even if it has changed.** If you want to **reload a module**, you have to restart the REPL. A package Revise exists that can keep your sessions running longer (see https://github.com/timholy/Revise.jl).


In [ ]:
include("wc.jl")

In [ ]:
using Main.LineCount

linecount("wc.jl")

### Command Objects for UNIX/LINUX/MacOS systems

For Windows users see https://docs.julialang.org/en/v1/manual/running-external-programs/#Running-External-Programs

A **shell commands** can be accessed through **shell-mode** in Julia by entering ; at the Julia promt and exit by _exit_:

```Julia
	julia>; <ENTER>

	shell> powershell
	.
	.
	.
	E:\Julia\...\code>exit
	julia>
```

Most operating systems provide a command-line interface, also known as a **`shell`**. Shells usually provide commands to navigate the file system and launch applications. For example, in Unix you can change directories with `cd`, display the contents of a directory with `ls`, and launch a web browser by typing (for example) firefox. 

On a UNIX/Linux system, **any program that you can launch from the shell can also be launched from Julia using a command object:**

```Julia
	julia> cmd = `echo hello`
		`echo hello`
```

**Backticks ` ` are used to delimit the command.** The function **`run`** executes the command:

```Julia
	julia> run(cmd);
		hello
```

The hello is the output of the echo command, sent to **`STDOUT`**. The **`run`** function itself returns a process object, and throws an ErrorException if the external command fails to run successfully. If you want to read the output of the **`external command`**, read can be used instead:

```Julia
	julia> a = read(cmd, String)
	    "hello\n"
```

For example, most Unix systems provide a command called md5sum or md5 that reads the contents of a file and computes a **checksum**. You can read about MD5 at https://en.wikipedia.org/wiki/Md5. This command provides an efficient way to check whether two files have the same contents. The probability that different contents yield the same checksum is very small (that is, unlikely to happen before the universe collapses). You can use a command object to run md5 from Julia and get the result:

```Julia
	julia> filename = "output.txt"
		"output.txt"

	julia> cmd = `md5 $filename`
		`md5 output.txt`

	julia> res = read(cmd, String)
		"MD5 (output.txt) = d41d8cd98f00b204e9800998ecf8427e\n"
```

### Debugging

When you are reading and writing files, you might run into **problems with whitespace**. These errors can be hard to debug because spaces, tabs and newlines are normally invisible:

```Julia
	julia> s = "1 2\t 3\n 4";

	julia> println(s)
		1 2     3
		 4
```

The built-in functions **repr** or **dump** can help. It takes any object as an argument and returns a string representation of the object.

```Julia
	julia> repr(s)
		"\"1 2\\t 3\\n 4\""

	julia> dump(s)
		String "1 2\t 3\n 4"
```

This can be helpful for debugging.

One other problem you might run into is that different systems use different characters to indicate the **end of a line**. Some systems use a **newline**, represented **\n**. Others use a **return character**, represented **\r**. Some use both. If you move files between different systems, these inconsistencies can cause problems. For most systems, there are applications to convert from one format to another. 

You can find them (and read more about this issue) at https://en.wikipedia.org/wiki/Newline and https://en.wikipedia.org/wiki/Whitespace_character.

## Exercises -- for UNIX/LINUX/MacOS users

### Exercise 14-2

Write a function called sed that takes as arguments a pattern string, a replacement string, and two filenames; it should read the first file and write the contents into the second file (creating it if necessary). If the pattern string appears anywhere in the file, it should be replaced with the replacement string.

If an error occurs while opening, reading, writing or closing files, your program should catch the exception, print an error message, and exit.

### Exercise 14-3

If you have done Exercise 12-3, you will see that a dictionary is created that maps from a sorted string of letters to the array of words that can be spelled with those letters. For example, "opst" maps to the array ["opts", "post", "pots", "spot", "stop", "tops"].

Write a module that imports anagramsets and provides two new functions: storeanagrams should store the anagram dictionary using JLD2; readanagrams should look up a word and return an array of its anagrams.

### Exercise 14-4

In a large collection of MP3 files, there may be more than one copy of the same song, stored in different directories or with different file names. The goal of this exercise is to search for duplicates.

Write a program that searches a directory and all of its subdirectories, recursively, and returns an array of complete paths for all files with a given suffix (like .mp3).

To recognize duplicates, you can use md5sum or md5 to compute a “checksum” for each files. If two files have the same checksum, they probably have the same contents.

To double-check, you can use the Unix command diff.